In [37]:
import pandas as pd
import numpy as np

pred_df = pd.read_csv('../data/predictions/ADA_predicted.csv',index_col='time')
pred_df.shape

window_df = pd.read_csv('../data/ready/ADA.csv', index_col='time').drop('Unnamed: 0', axis = 1).drop_duplicates()
window_df.head(50)

window_df.head(50)

df_suffixed = window_df.add_suffix('_df')
pred_df_suffixed = pred_df.add_suffix('_pred_df')
df = df_suffixed.join(pred_df_suffixed, how='outer')

def backtest(df, capital=1000, leverage=1, fee_pct=0, ticker = ticker):
    trade_history = []
    current_capital = capital

    for i in range(len(df)):
        # If met_threshold_pred_df is 0, skip the trade
        if df['met_threshold_pred_df'].iloc[i] == 0:
            continue

        # If lag_time_pred_df is 0 or NaN, skip the trade
        lag_time = df['lag_time_pred_df'].iloc[i]
        if pd.isnull(lag_time) or lag_time == 0:
            continue

        # If lag_time_pred_df would cause us to go beyond the DataFrame, skip the trade
        lag_time = int(lag_time)  # Ensure lag_time is an integer
        if i + lag_time >= len(df):
            continue

        # Get the start and end price
        start_price = df[f'{ticker}_close_df'].iloc[i]
        end_price = df[f'{ticker}_close_df'].iloc[i + lag_time]

        # Get the opening and closing times for the trade
        open_time = df.index[i]
        close_time = df.index[i + lag_time]

        # Calculate percentage change in price
        pct_change = (end_price - start_price) / start_price

        # Account for leverage
        pct_change *= leverage

        # Deduct trading fees
        pct_change -= fee_pct/100  # Convert fee_pct to a fraction

        # Calculate profit/loss
        pnl = current_capital * pct_change

        # Update capital
        current_capital += pnl

        # Record trade details
        trade_history.append({
            'open_time': open_time,
            'close_time': close_time,
            'start_price': start_price,
            'end_price': end_price,
            'pct_change': pct_change,
            'pnl': pnl,
            'capital': current_capital
        })

    return pd.DataFrame(trade_history)


backtest(df, capital=1000, leverage=1, fee_pct=0, ticker = 'ADA')

csv_files = {
    "ADA": "../data/merged_data/ADA.csv",
    "BNB": "../data/merged_data/BNB.csv",
    "DOGE": "../data/merged_data/DOGE.csv",
    #"DOT": "../data/merged_data/DOT.csv",
    "ETH": "../data/merged_data/ETH.csv",
    "LINK": "../data/merged_data/LINK.csv",
    "LTC": "../data/merged_data/LTC.csv",
    "UNI": "../data/merged_data/UNI.csv",
    "XRP": "../data/merged_data/XRP.csv"
}

tickers = list(csv_files.keys())


for ticker in tickers:
    pred_df = pd.read_csv(f'../data/predictions/{ticker}_predicted.csv',index_col='time')
    window_df = pd.read_csv(f'../data/ready/{ticker}.csv', index_col='time').drop('Unnamed: 0', axis = 1).drop_duplicates()
    df_suffixed = window_df.add_suffix('_df')
    pred_df_suffixed = pred_df.add_suffix('_pred_df')
    df = df_suffixed.join(pred_df_suffixed, how='outer')

    portfolio = backtest(df, capital=1000, leverage=1, fee_pct=0, ticker = ticker)
    portfolio.to_csv(f'../data/portfolio-performance/{ticker}_portfolio.csv')

pd.read_csv('../data/portfolio-performance/DOGE_portfolio.csv')